# 📋 Business Requirement Chatbot with Feature Tracking & Word Export

This notebook helps translate plain English or meeting audio into structured business requirements, with support for tagging additional features and exporting to a formatted Word document.

## 📦 Step 1: Install Required Libraries

In [ ]:
!pip install gradio openai-whisper python-docx

## 🧠 Step 2: Define Requirement Conversion Logic and Export

In [ ]:

import re
from docx import Document
from datetime import datetime

history = []

def convert_to_business_requirement(text, label="Core Feature"):
    requirements = []
    sentences = re.split(r'[.?!]', text)
    for sentence in sentences:
        if len(sentence.strip()) > 0:
            requirement = f"- [{label}] The system shall {sentence.strip().lower()}."
            requirements.append(requirement)
    history.extend(requirements)
    return "\n".join(requirements)

def show_history():
    return "\n".join(history)

def export_to_word():
    doc = Document()
    doc.add_heading("Business Requirements Export", 0)
    doc.add_paragraph(f"Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    for req in history:
        doc.add_paragraph(req, style='List Bullet')
    path = "/mnt/data/Business_Requirements_Export.docx"
    doc.save(path)
    return path
    

## 🔊 Step 3: Transcribe Meeting Audio

In [ ]:

import whisper
model = whisper.load_model("base")

def transcribe_audio(audio_file, label="Core Feature"):
    result = model.transcribe(audio_file.name)
    return convert_to_business_requirement(result["text"], label)
    

## 🤖 Step 4: Gradio Interface with Export Button

In [ ]:

import gradio as gr

text_input = gr.Interface(
    fn=lambda text, label: convert_to_business_requirement(text, label),
    inputs=[
        gr.Textbox(label="Describe a business process or requirement", lines=4),
        gr.Textbox(label="Feature Label (e.g. Core Feature, Additional Feature)", value="Core Feature")
    ],
    outputs=gr.Textbox(label="Generated Requirements"),
    title="🧠 Text to Business Requirement",
    description="Describe new features and assign a label to track core or additional functionality."
)

audio_input = gr.Interface(
    fn=lambda audio, label: transcribe_audio(audio, label),
    inputs=[
        gr.Audio(type="filepath", label="Upload meeting audio"),
        gr.Textbox(label="Feature Label (e.g. Core Feature, Additional Feature)", value="Core Feature")
    ],
    outputs=gr.Textbox(label="Generated Requirements from Audio"),
    title="🎙️ Audio to Requirement Generator",
    description="Upload meeting clips and tag input as Core or Additional Features."
)

history_display = gr.Interface(
    fn=show_history,
    inputs=[],
    outputs=gr.Textbox(label="📜 Requirement History Log"),
    title="📚 Full Requirement Log",
    description="View all business requirements, organized by feature tag."
)

export_docx = gr.Interface(
    fn=lambda: export_to_word(),
    inputs=[],
    outputs=gr.File(label="Download Requirements Docx"),
    title="📤 Export to Word",
    description="Download all formatted business requirements in Word format."
)

gr.TabbedInterface(
    [text_input, audio_input, history_display, export_docx],
    ["Text Input", "Audio Input", "View All Requirements", "Export to Word"]
).launch()
    